In [1]:
# !pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from csv import reader
from pyspark.sql.types import DoubleType,StringType,IntegerType,DateType
from pyspark.sql.functions import col,when,count,desc
from pyspark.sql.functions import format_string
import sys
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
sc=SparkContext.getOrCreate()
spark = SparkSession(sc)

ratings = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
          .option("database", "IPL").option("collection", "ratings").load()


In [2]:
ratings=ratings.drop("_id")
ratings.dtypes
ratings=ratings.withColumnRenamed("label","type")
ratings.dtypes

Out[5]: [('Batting_Hand', 'string'),
 ('Bowling_Skill', 'string'),
 ('Buy_Type', 'string'),
 ('EconomyRate', 'double'),
 ('Player_Name', 'string'),
 ('RatingNorm', 'double'),
 ('StrikeRate', 'double'),
 ('age', 'int'),
 ('type', 'string'),
 ('runs', 'bigint'),
 ('wickets', 'bigint')]

In [3]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler

from distutils.version import LooseVersion

categoricalColumns = ["Batting_Hand", "Bowling_Skill", "Player_Name", "type"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    if LooseVersion(pyspark.__version__) < LooseVersion("3.0"):
        from pyspark.ml.feature import OneHotEncoderEstimator
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    else:
        from pyspark.ml.feature import OneHotEncoder
        encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol="Buy_Type", outputCol="label")
stages += [label_stringIdx]

In [4]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "EconomyRate", "RatingNorm", "StrikeRate", "runs", "wickets"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [5]:
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(ratings)
preppedDataDF = pipelineModel.transform(ratings)


In [6]:
preppedDataDF.select(["Buy_Type","label"]).show()

+---------+-----+
 Buy_Type|label|
+---------+-----+
 Avoid| 0.0|
Excellent| 3.0|
 Avoid| 0.0|
 Maybe| 1.0|
 Avoid| 0.0|
 Avoid| 0.0|
 Good Buy| 2.0|
 Avoid| 0.0|
 Avoid| 0.0|
 Good Buy| 2.0|
Excellent| 3.0|
 Avoid| 0.0|
 Avoid| 0.0|
 Avoid| 0.0|
 Avoid| 0.0|
 Avoid| 0.0|
 Maybe| 1.0|
 Avoid| 0.0|
 Maybe| 1.0|
 Avoid| 0.0|
+---------+-----+
only showing top 20 rows

In [7]:
(trainingData, testData) = preppedDataDF.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

322
147

In [8]:
from pyspark.ml.classification import RandomForestClassifier
# from pyspark.mllib.tree import RandomForest, RandomForestModel

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")
# model = RandomForest.trainClassifier(trainingData, numClasses=4,categoricalFeaturesInfo={},numTrees=3, featureSubsetStrategy="auto",impurity='gini', maxDepth=4, maxBins=32)

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [9]:
predictions = rfModel.transform(testData)
predictions.printSchema()
# RandomForestClassificationModel.transform

root
-- Batting_Hand: string (nullable = true)
-- Bowling_Skill: string (nullable = true)
-- Buy_Type: string (nullable = true)
-- EconomyRate: double (nullable = false)
-- Player_Name: string (nullable = true)
-- RatingNorm: double (nullable = false)
-- StrikeRate: double (nullable = false)
-- age: integer (nullable = true)
-- type: string (nullable = true)
-- runs: long (nullable = true)
-- wickets: long (nullable = true)
-- Batting_HandIndex: double (nullable = false)
-- Batting_HandclassVec: vector (nullable = true)
-- Bowling_SkillIndex: double (nullable = false)
-- Bowling_SkillclassVec: vector (nullable = true)
-- Player_NameIndex: double (nullable = false)
-- Player_NameclassVec: vector (nullable = true)
-- typeIndex: double (nullable = false)
-- typeclassVec: vector (nullable = true)
-- label: double (nullable = false)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [10]:
selected = predictions.select("label", "prediction", "probability", "Player_Name", "Buy_Type")
display(selected)

label,prediction,probability,Player_Name,Buy_Type
0.0,0.0,"List(1, 4, List(), List(0.7799185243054638, 0.15632476450814703, 0.04737766328288053, 0.016379047903508675))",WPUJC Vaas,Avoid
2.0,2.0,"List(1, 4, List(), List(0.1848933440203622, 0.24904381185338637, 0.364139117914291, 0.20192372621196036))",MJ McClenaghan,Good Buy
2.0,2.0,"List(1, 4, List(), List(0.11519023399114783, 0.23931458351279292, 0.461955953878161, 0.18353922861789826))",Mustafizur Rahman,Good Buy
2.0,2.0,"List(1, 4, List(), List(0.1809347859227832, 0.2879938063854808, 0.3762644014590123, 0.15480700623272378))",BE Hendricks,Good Buy
1.0,0.0,"List(1, 4, List(), List(0.5639539573088351, 0.38389202493395497, 0.04257107492077839, 0.009582942836431418))",BB Sran,Maybe
0.0,0.0,"List(1, 4, List(), List(0.6658635117219838, 0.2469881602195348, 0.055053010007463245, 0.032095318051018215))",S Aravind,Avoid
0.0,0.0,"List(1, 4, List(), List(0.8242512748882145, 0.13104650110488364, 0.032954586359803605, 0.011747637647098422))",SB Wagh,Avoid
2.0,2.0,"List(1, 4, List(), List(0.254673071932518, 0.24800453802262518, 0.3297195851431966, 0.16760280490166013))",Sohail Tanvir,Good Buy
0.0,0.0,"List(1, 4, List(), List(0.7706378225510779, 0.17610546626253304, 0.04021099661621387, 0.013045714570175346))",S Chanderpaul,Avoid
2.0,1.0,"List(1, 4, List(), List(0.22478539406660528, 0.3173053595178476, 0.31240298956457646, 0.14550625685097063))",PP Chawla,Good Buy


--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3235231520490796> in <module> 
 6 .build())# Iinitializing the cross validator class
 7 cv = CrossValidator ( estimator = rfModel , estimatorParamMaps = paramGrid , evaluator = evaluator , numFolds = 5 ) # Run cross validations. This can take about 6 minutes since it is training over 20 trees 
 ----> 8 cvModel = cv . fit ( trainingData ) 
 9 gbt_predictions_2 = cvModel . transform ( testData ) 
 10 my_eval . evaluate ( gbt_predictions_2 ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 307 train = df . filter ( ~ condition ) . cache ( ) 
 308 
 --> 309 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 310 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 311 metrics [ j ] += ( metric / nFolds ) 

 /databricks/spark/python/pyspark/ml/tuning.py in _parallelFitTasks (est, train, eva, validation, epm, collectSubModel) 
 49 : return : ( int , float , subModel ) , an index into ` epm ` and the associated metric value . 
 50 """
 ---> 51 modelIter = est . fitMultiple ( train , epm ) 
 52 
 53 def singleTask ( ) : 

 AttributeError : 'RandomForestClassificationModel' object has no attribute 'fitMultiple'

In [12]:
predictions.select("prediction").distinct().show()

+----------+
prediction|
+----------+
 0.0|
 1.0|
 3.0|
 2.0|
+----------+

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[22]: 0.7339531092351341

In [14]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5,5))
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(lrModel.summary.roc.select('FPR').collect(),
         lrModel.summary.roc.select('TPR').collect())
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [15]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("label","Player_Name","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+-----+--------------+------------------------------+-----+----------+
label| Player_Name| probability|label|prediction|
+-----+--------------+------------------------------+-----+----------+
 0.0| WPUJC Vaas|[1.0,1.4258225188181316E-17...| 0.0| 0.0|
 0.0|AD Mascarenhas|[1.0,7.835381960566884E-18,...| 0.0| 0.0|
 1.0| SP Fleming|[1.0,4.4928465304671634E-29...| 1.0| 0.0|
 2.0| MJ Guptill|[1.0,1.513437921952876E-47,...| 2.0| 0.0|
 0.0| Mohammad Nabi|[0.9999999999999993,7.70382...| 0.0| 0.0|
 2.0| DL Chahar|[0.999999999999962,1.123109...| 2.0| 0.0|
 1.0| N Pooran|[0.9999999999685409,3.14591...| 1.0| 0.0|
 0.0| DJG Sammy|[0.9999999983113883,1.45345...| 0.0| 0.0|
 2.0| MR Marsh|[0.9999999902824385,1.14263...| 2.0| 0.0|
 2.0| AJ Tye|[0.9999999097498443,4.17497...| 2.0| 0.0|
+-----+--------------+------------------------------+-----+----------+
only showing top 10 rows

In [16]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[74]: 0.783269371809594